Importing necessary libraries

In [20]:
import os
import random
from datetime import datetime

Here we declare the txt files path as well as names for 'accounts' as well as 'transactions'. They're the one storing credentials as well as transaction details.

In [22]:
# File paths
ACCOUNTS_FILE = "accounts.txt"
TRANSACTIONS_FILE = "transactions.txt"

Admin credentials declared.

In [24]:
ADMIN_USERNAME = "admin"
ADMIN_PASSWORD = "admin@1234"

Check and create the txt files.

In [26]:
def initialize_files():
    for file in [ACCOUNTS_FILE, TRANSACTIONS_FILE]:
        if not os.path.exists(file):
            with open(file, "w") as f:
                pass  # Create empty file

In [27]:
def write_to_file(filename, data):
    with open(filename, "a") as file:
        file.write(data + "\n")

def read_file_lines(filename):
    if not os.path.exists(filename):
        return []
    with open(filename, "r") as file:
        return file.readlines()

In [28]:
def generate_unique_account_number():
    existing_account_numbers = {line.split(",")[0] for line in read_file_lines(ACCOUNTS_FILE)}
    while True:
        account_number = str(random.randint(100000, 999999))  # This will generate the user ID/account number
        if account_number not in existing_account_numbers:
            return account_number

In [29]:
def create_account():
    name = input("Enter user's name: ")
    initial_deposit = float(input("Enter initial deposit: "))
    account_number = generate_unique_account_number()
    password = input("Enter user's password: ")

    account_data = f"{account_number},{name},{password},{initial_deposit}"
    write_to_file(ACCOUNTS_FILE, account_data)
    print(f"Account created successfully! Account number: {account_number}")

def delete_account():
    account_number = input("Enter account number to delete: ")
    accounts = read_file_lines(ACCOUNTS_FILE)
    with open(ACCOUNTS_FILE, "w") as file:
        for account in accounts:
            acc_no, _, _, _ = account.strip().split(",")
            if acc_no != account_number:
                file.write(account)
            else:
                print(f"Account {account_number} deleted successfully!")

In [30]:
def login(account_number, password):
    accounts = read_file_lines(ACCOUNTS_FILE)
    for account in accounts:
        acc_no, name, acc_pass, balance = account.strip().split(",")
        if acc_no == account_number and acc_pass == password:
            return name, float(balance)
    return None, None

Here we add the feature of view balance.

In [32]:
def view_current_balance(account_number):
    accounts = read_file_lines(ACCOUNTS_FILE)
    for account in accounts:
        acc_no, name, acc_pass, balance = account.strip().split(",")
        if acc_no == account_number:
            print(f"Your current balance is: {balance}")
            return
    print("Account not found.")

def update_account_balance(account_number, new_balance):
    """Update the balance of the given account number in accounts.txt."""
    accounts = read_file_lines(ACCOUNTS_FILE)
    with open(ACCOUNTS_FILE, "w") as file:
        for account in accounts:
            acc_no, name, acc_pass, balance = account.strip().split(",")
            if acc_no == account_number:
                balance = str(new_balance)
            file.write(f"{acc_no},{name},{acc_pass},{balance}\n")

Here we have code snippet to perform & log transaction.

In [34]:
def perform_transaction(account_number, balance):
    print("\n1. Deposit")
    print("2. Withdraw")
    print("3. Transfer to another account")
    choice = int(input("Enter your choice: "))

    amount = float(input("Enter amount: "))
    if choice == 1:  # Deposit
        balance += amount
        transaction_type = "Deposit"
        print(f"Deposit successful! Current balance: {balance}")
    elif choice == 2:  # Withdrawal
        if amount > balance:
            print("Insufficient funds.")
            return balance
        balance -= amount
        transaction_type = "Withdrawal"
        print(f"Withdrawal successful! Current balance: {balance}")
    elif choice == 3:  # Transfer
        recipient_account = input("Enter recipient account number: ")
        if amount > balance:
            print("Insufficient funds.")
            return balance
        accounts = read_file_lines(ACCOUNTS_FILE)
        recipient_found = False
        for account in accounts:
            acc_no, name, acc_pass, acc_balance = account.strip().split(",")
            if acc_no == recipient_account:
                recipient_balance = float(acc_balance) + amount
                recipient_found = True
                break
        if not recipient_found:
            print("Recipient account not found.")
            return balance
        balance -= amount
        print(f"Transfer successful! Remaining balance: {balance}")

        # Update recipient account
        with open(ACCOUNTS_FILE, "w") as file:
            for account in accounts:
                acc_no, name, acc_pass, acc_balance = account.strip().split(",")
                if acc_no == account_number:
                    acc_balance = str(balance)
                elif acc_no == recipient_account:
                    acc_balance = str(recipient_balance)
                file.write(f"{acc_no},{name},{acc_pass},{acc_balance}\n")
        transaction_type = "Transfer"
    else:
        print("Invalid choice.")
        return balance

    # Log transaction
    transaction_data = f"{account_number},{transaction_type},{amount},{datetime.now().date()}"
    write_to_file(TRANSACTIONS_FILE, transaction_data)

    # Update balance in accounts.txt
    update_account_balance(account_number, balance)

    return balance

Now we create the interface for the main menu.

In [36]:
def main():
    initialize_files()  # Ensure files exist

    while True:
        print("\nWelcome to the Banking System!")
        user_id = input("Enter User ID: ")
        password = input("Enter Password: ")

        if user_id == ADMIN_USERNAME and password == ADMIN_PASSWORD:
            print("\nAdmin Mode")
            while True:
                print("\n1. Create Account")
                print("2. Delete Account")
                print("3. Logout")
                choice = int(input("Enter your choice: "))
                if choice == 1:
                    create_account()
                elif choice == 2:
                    delete_account()
                elif choice == 3:
                    print("Logged out of Admin Mode.")
                    break
                else:
                    print("Invalid choice.")
        else:
            name, balance = login(user_id, password)
            if name:
                print(f"\nLogin successful! Welcome, {name}")
                while True:
                    print("\n1. View Current Balance")
                    print("2. Perform Transaction")
                    print("3. Logout")
                    choice = int(input("Enter your choice: "))
                    if choice == 1:
                        view_current_balance(user_id)
                    elif choice == 2:
                        balance = perform_transaction(user_id, balance)
                    elif choice == 3:
                        print("Logged out successfully!")
                        break
                    else:
                        print("Invalid choice.")
            else:
                print("Invalid User ID or Password.")

if __name__ == "__main__":
    main()


Welcome to the Banking System!


Enter User ID:  379297
Enter Password:  justin



Login successful! Welcome, Justin

1. View Current Balance
2. Perform Transaction
3. Logout


Enter your choice:  2



1. Deposit
2. Withdraw
3. Transfer to another account


...

Enter your choice:  2



1. Deposit
2. Withdraw
3. Transfer to another account


Enter your choice:  3
Enter amount:  5000
Enter recipient account number:  905609


Transfer successful! Remaining balance: 223432.0

1. View Current Balance
2. Perform Transaction
3. Logout


KeyboardInterrupt: Interrupted by user